<a href="https://colab.research.google.com/github/CheMLers/covid-19/blob/master/COVID_19_Nishank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
pip install -U keras



---


**Internation Time Series data**


---



In [0]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import math
#import tensorflow as tf
#from tensorflow.python.keras.layers import Dense, LSTM
#from tensorflow.python.keras import Sequential
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
np.random.seed(7)

# source = https://github.com/datasets/covid-19/edit/master/data/time-series-19-covid-combined.csv

data = pd.read_csv('/content/drive/My Drive/google collab/COVID-19_Nishank/data.csv')
data = data[["Date","Country/Region","Province/State","Confirmed","Recovered","Deaths"]]
aggregated_data = data.groupby(["Date"]).sum()
india_data = data[data["Country/Region"] == "India"]
india_data = india_data.groupby(["Date"]).sum()
italy_data = data[data["Country/Region"] == "Italy"]
italy_data = italy_data.groupby(["Date"]).sum()

data = aggregated_data["Confirmed"]
dataX = dataY = data
dataX = dataX[:-1]
dataY = dataY[1:]
dataX = dataX.to_numpy()
dataY = dataY.to_numpy()
dataX = dataX.reshape(len(dataY),1)
dataY = dataY.reshape(len(dataY),1)

trainX = dataX[:int(0.9*len(dataX))]
testX = dataX[int(0.9*len(dataX)):]
trainY = dataY[:int(0.9*len(dataX))]
testY = dataY[int(0.9*len(dataX)):] 

scalerX=MinMaxScaler(feature_range=(0,1))
trainX = scalerX.fit_transform(trainX)
testX = scalerX.transform(testX)
scalerY=MinMaxScaler(feature_range = (0,1))
trainY = scalerY.fit_transform(trainY)
testY = scalerY.transform(testY)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

model = Sequential()
model.add(LSTM(units = 256, input_shape=(1, 1)))
#model.add(Dense(units = 4))
model.add(Dense(units = 128))
model.add(Dense(units = 64))
model.add(Dense(units = 32))
model.add(Dense(units = 1, activation = "linear"))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=200, batch_size=1, verbose=0 , validation_split=0.1)

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scalerY.inverse_transform(trainPredict)
trainY = scalerY.inverse_transform(trainY)
testPredict = scalerY.inverse_transform(testPredict)
testY = scalerY.inverse_transform(testY)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))


# shift train predictions for plotting
trainPredictPlot = np.empty((len(dataY)+1,1))
trainPredictPlot[:,0] = np.nan
trainPredictPlot[0:len(trainPredict),0] = trainPredict[:,0]
# shift test predictions for plotting
testPredictPlot = np.empty((len(dataY)+1,1))
testPredictPlot[:,0] = np.nan
testPredictPlot[len(trainPredict):len(dataY),0] = testPredict[:,0]
# plot baseline and predictions
plt.plot(dataY)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()



---

**India Data Analysis for best architecture till now**

---



In [0]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import math
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
np.random.seed(7)

# source = https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#

data = pd.read_csv('/content/drive/My Drive/google collab/COVID-19_Nishank/India_data.txt', delimiter = '\s+')

data_confirmed = data[data["Status"] == "Confirmed"]
data_confirmed = data_confirmed.drop(["Date","Status"],axis = 1)
data_confirmed = data_confirmed.reset_index()
data_confirmed = data_confirmed.drop(["index"],axis = 1)
data_confirmed = data_confirmed.fillna(0)
data_confirmed_cum = data_confirmed.cumsum()
#data_confirmed_cum = data_confirmed_cum.stack().groupby(level=0).nlargest(10).unstack().reset_index(level=1, drop=True).reindex(columns=data_confirmed_cum.columns).fillna(0)

#dataX = data_confirmed_cum.drop(["TT"],axis = 1)
dataX = data_confirmed_cum["TT"]
dataY = data_confirmed_cum["TT"]
dataX = dataX[:-1]
dataY = dataY[1:]
dataX = dataX.to_numpy()
dataY = dataY.to_numpy()
dataX = dataX.reshape(len(dataY),1)
dataY = dataY.reshape(len(dataY),1)

trainX = dataX[:int(0.8*len(dataX))]
testX = dataX[int(0.8*len(dataX)):]
trainY = dataY[:int(0.8*len(dataX))]
testY = dataY[int(0.8*len(dataX)):] 

scalerX=MinMaxScaler(feature_range=(0,1))
trainX = scalerX.fit_transform(trainX)
testX = scalerX.transform(testX)
scalerY=MinMaxScaler(feature_range = (0,1))
trainY = scalerY.fit_transform(trainY)
testY = scalerY.transform(testY)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

model = Sequential()
model.add(LSTM(units = 256, input_shape=(1, 1)))
model.add(Dense(units = 128))
model.add(Dense(units = 64))
model.add(Dense(units = 32))
model.add(Dense(units = 1, activation = "linear"))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=200, batch_size=1, verbose=0 , validation_split=0.1)
model.save("/content/drive/My Drive/google collab/COVID-19_Nishank/best_india_model.h5")
#model = load_model('/content/drive/My Drive/google collab/COVID-19_Nishank/best_india_model.h5')
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scalerY.inverse_transform(trainPredict)
trainY = scalerY.inverse_transform(trainY)
testPredict = scalerY.inverse_transform(testPredict)
testY = scalerY.inverse_transform(testY)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))


# shift train predictions for plotting
trainPredictPlot = np.empty((len(dataY)+1,1))
trainPredictPlot[:,0] = np.nan
trainPredictPlot[0:len(trainPredict),0] = trainPredict[:,0]
# shift test predictions for plotting
testPredictPlot = np.empty((len(dataY)+1,1))
testPredictPlot[:,0] = np.nan
testPredictPlot[len(trainPredict):len(dataY),0] = testPredict[:,0]
# plot baseline and predictions
plt.plot(dataY)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()




---
**Code with predictor function for future days as well**


---




In [0]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import math
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
np.random.seed(7)

def predictor(data,days,split):
  dataX = data
  dataY = data

  dataX = dataX[:-1]
  dataY = dataY[1:]
  dataX = dataX.to_numpy()
  dataY = dataY.to_numpy()
  dataX = dataX.reshape(len(dataY),1)
  dataY = dataY.reshape(len(dataY),1)

  trainX = dataX[:int(split*len(dataX))]
  testX = dataX[int(split*len(dataX)):]
  trainY = dataY[:int(split*len(dataX))]
  testY = dataY[int(split*len(dataX)):] 

  scalerX=MinMaxScaler(feature_range=(0,1))
  trainX = scalerX.fit_transform(trainX)
  testX = scalerX.transform(testX)
  scalerY=MinMaxScaler(feature_range = (0,1))
  trainY = scalerY.fit_transform(trainY)
  testY = scalerY.transform(testY)

  trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
  testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

  model = Sequential()
  model.add(LSTM(units = 256, input_shape=(1, 1)))
  model.add(Dense(units = 128))
  model.add(Dense(units = 64))
  model.add(Dense(units = 32))
  model.add(Dense(units=16))
  model.add(Dense(units = 1, activation = "linear"))
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(trainX, trainY, epochs=200, batch_size=1, verbose=0 , validation_split=0.2)

  plt.figure()
  plt.title("Training Data only")
  trainPredict = scalerY.inverse_transform(model.predict(trainX))
  predict = model.predict(testX)
  testPredict = scalerY.inverse_transform(predict)

  trainPredictPlot = np.empty((len(dataY),1))
  trainPredictPlot[:,0] = np.nan
  trainPredictPlot[0:len(trainPredict),0] = trainPredict[:,0]
  # shift test predictions for plotting
  testPredictPlot = np.empty((len(dataY)+1,1))
  testPredictPlot[:,0] = np.nan
  testPredictPlot[len(trainPredict):len(dataY),0] = testPredict[:,0]
  # plot baseline and predictions
  plt.plot(dataY)
  plt.plot(trainPredictPlot)
  plt.plot(testPredictPlot)
  plt.show()

  model.fit(testX,testY,epochs = 30, batch_size = 1, verbose=0,validation_split = 0.1)

  plt.figure()
  plt.title("Full Data")
  trainPredict = scalerY.inverse_transform(model.predict(trainX))
  predict = model.predict(testX)
  testPredict = scalerY.inverse_transform(predict)

  trainPredictPlot = np.empty((len(dataY),1))
  trainPredictPlot[:,0] = np.nan
  trainPredictPlot[0:len(trainPredict),0] = trainPredict[:,0]
  # shift test predictions for plotting
  testPredictPlot = np.empty((len(dataY),1))
  testPredictPlot[:,0] = np.nan
  testPredictPlot[len(trainPredict):len(dataY),0] = testPredict[:,0]
  # plot baseline and predictions
  plt.plot(dataY)
  plt.plot(trainPredictPlot)
  plt.plot(testPredictPlot)

  futurePredictPlot = np.empty(len(dataY)-1)
  futurePredictPlot[:] = np.nan
  futurePredictPlot = np.append(futurePredictPlot,testPredict[-1:,0])

  for i in range(days):
    future_pred = dataY[-1:]
    future_pred = np.reshape(future_pred,(1,1))
    future_pred = scalerX.transform(future_pred)
    future_pred = np.reshape(future_pred, (future_pred.shape[0], 1, future_pred.shape[1]))
    futurePredictPlot = np.append(futurePredictPlot,scalerY.inverse_transform(model.predict(future_pred)))
    print(scalerY.inverse_transform(model.predict(future_pred)))
    trainX = future_pred
    trainY = np.reshape(model.predict(future_pred),(1,1))
    model.fit(trainX,trainY,epochs = 2, verbose = 0)
    dataY = np.append(dataY,scalerY.inverse_transform(model.predict(future_pred)))
  
  plt.plot(futurePredictPlot)


In [0]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import math

# source = https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#

data = pd.read_csv('/content/drive/My Drive/google collab/COVID-19_Nishank/India_data.txt', delimiter = '\s+')

data_confirmed = data[data["Status"] == "Confirmed"]
data_confirmed = data_confirmed.drop(["Date","Status"],axis = 1)
data_confirmed = data_confirmed.reset_index()
data_confirmed = data_confirmed.drop(["index"],axis = 1)
data_confirmed = data_confirmed.fillna(0)
data_confirmed_cum = data_confirmed.cumsum()
predictor(data_confirmed_cum["TT"],1,0.8)



---
**Predictor function with added parameter of lookback**


---






---

**Growth rate vs Confirmed cases Parabolic analysis**



---



In [0]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import math
#import tensorflow as tf
#from tensorflow.python.keras.layers import Dense, LSTM
#from tensorflow.python.keras import Sequential
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
np.random.seed(7)

# source = https://github.com/datasets/covid-19/edit/master/data/time-series-19-covid-combined.csv

data = pd.read_csv('/content/drive/My Drive/google collab/COVID-19_Nishank/data.csv')
data = data[["Date","Country/Region","Province/State","Confirmed","Recovered","Deaths"]]
aggregated_data = data.groupby(["Date"]).sum()
india_data = data[data["Country/Region"] == "India"]
india_data = india_data.groupby(["Date"]).sum()
italy_data = data[data["Country/Region"] == "Italy"]
italy_data = italy_data.groupby(["Date"]).sum()
#italy_data = aggregated_data
italy_data = italy_data["Confirmed"]
italy_data_diff = italy_data.diff()
#plt.plot(italy_data,italy_data_diff)
dataX = italy_data
dataY = italy_data_diff
dataX = dataX[:-1]
dataY = dataY[1:]
dataX = dataX.to_numpy()
dataY = dataY.to_numpy()

trainX = dataX[:int(0.9*len(dataX))]
testX = dataX[int(0.9*len(dataX)):]
trainY = dataY[:int(0.9*len(dataX))]
testY = dataY[int(0.9*len(dataX)):] 

a = np.polyfit(trainX, trainY, 2)
b = np.poly1d(a)

plt.figure(1)
plt.plot(dataY)
plt.plot(b(dataX))
#plt.plot(b(testX))

plt.figure(2)
plt.plot(dataY.cumsum())
plt.plot(b(dataX).cumsum())